# Code for generating and running PH of noisy networks - dev

In [ ]:
# Need to load in all the network generation functions here?


In [2]:
# Load packages

using Pkg
using Statistics
using Plots
using LinearAlgebra
using Distances
using Eirene
using StatsBase
using Random
using Distributions
using JLD
using MAT

include("graph_models.jl")
include("helper_functions.jl")


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-10289121054137075192\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9453891260566711732\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-10289121054137075192\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.2/Eirene/mCiG1.ji for Eirene [9c0f25c4-2ca1-5870-89f6-52640788da1d]
└ @ Base loading.jl:1240


betticurveFromBarcode (generic function with 1 method)

## Choose weighed network and generate replicates

--------------------------------
## CHANGE PARAMETERS HERE
---------------------------------

In [3]:
## Pick from one of the graph models and generate out to THRESH edge density

nReps = 50
rho = 0.35      # Threshold at edge density = rho
nNodes = 70
save_figures = 0  # Boolean to save figure pdfs
save_data = 0    # Boolean to save data  

# for geometricConf
p = 0.01
scale_factor = 100

# for RG
dims = 3

# for discreteUniformConf
a = 0
b = 1000

# for cosineGeometric
dims = 3


# Graph model name one of "geometricConf", "IID" , "RG", "discreteUniformConf", "cosineGeometric", "RL", "assoc"
# "disassort", "coreperiph", "DP"
graph_model_name = IID"

LoadError: syntax: incomplete: invalid string syntax

In [4]:


weighted_graph_array = zeros(nNodes,nNodes,nReps)
weighted_graph_array_ord = zeros(nNodes,nNodes,nReps)
betti_file_name = []
parameters = []
for rep in 1:nReps
    
    
    if graph_model_name == "geometricConf"
        G_i = make_dev_Geometric_configuration_model(nNodes,p,scale_factor)
        parameters = [nReps, rho, nNodes, p, scale_factor]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "IID"
        G_i = make_iid_weighted_graph(nNodes)
        parameters = [nReps, rho, nNodes]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "RG"
        G_i = make_random_geometric(nNodes,dims)
        parameters = [nReps, rho, nNodes, dims]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
       
        
    elseif graph_model_name == "discreteUniformConf"
        G_i = make_dev_DiscreteUniform_configuration_model(nNodes,a,b)
        parameters = [nReps, rho, nNodes, a, b]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "cosineGeometric"
        G_i = make_cosine_geometric(nNodes,dims)
        parameters = [nReps, rho, nNodes, dims]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "RL"
        G_i = make_ring_lattice_wei(nNodes)
        parameters = [nReps, rho, nNodes]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "assoc"
        
        mat_dict = matread("../data/associative_WSBM_70_10_10_2_2_09_05.mat")
        parameters = [mat_dict["param_array"]; rho]
        G_i = mat_dict["adj_array"][:,:,rep]
        G_i = G_i+transpose(G_i)

        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
    elseif graph_model_name == "coreperiph"
        
        mat_dict = matread("../data/coreperiph_WSBM_70_10_10_2_2_09_05.mat")
        parameters = [mat_dict["param_array"]; rho]
        G_i = mat_dict["adj_array"][:,:,rep]
        G_i = G_i+transpose(G_i)

        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "disassort"
        # These wsbm come out asymmetric. We don't need to worry about averaging the weights, just add the transpose.
        mat_dict = matread("../data/disassortative_WSBM_70_10_10_2_2_09_05.mat")
        parameters = [mat_dict["param_array"]; rho]
        G_i = mat_dict["adj_array"][:,:,rep]
        G_i = G_i+transpose(G_i)

        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "DP"
        G_i = make_dot_product(nNodes,dims)
        parameters = [nReps, rho, nNodes, dims]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
    end
    
        

    
    # Threshold at rho
    G_i_thresh = threshold_graph(G_i,rho,nNodes)
    
    weighted_graph_array[:,:,rep] = G_i_thresh
    
    
    edge_list_ranks = denserank([G_i...], rev = true)   # so highest edge weight gets assigned 1
    G_i_ord = reshape(edge_list_ranks,(nNodes,nNodes))
    G_i_ord[diagind(G_i_ord)] .= 0
    weighted_graph_array_ord[:,:,rep] = G_i_ord
    
    
 
    
end
println("Naming files $(betti_file_name)")

# We can add noise to the entire array at the same time

weighted_graph_array_iidNoise = deepcopy(weighted_graph_array)
weighted_graph_array_iidNoise_ord = zeros(nNodes,nNodes,nReps)
for rep in 1:nReps
    
    G_i = weighted_graph_array_iidNoise[:,:,rep]
    G_i[G_i.>0] .= G_i[G_i .>0 ] .+1
    
    # Now the real values are all >1, so we can add noise which will be < 1
    
    G_rand = make_iid_weighted_graph(nNodes)
    G_i_rand = deepcopy(G_i)
    G_i_rand[G_i_rand .== 0] .= G_rand[G_i_rand .== 0]
    
    # So anything <1 in G_i_rand will be from random noise, and any entry >1 will be real matrix
    
    weighted_graph_array_iidNoise[:,:,rep] = G_i_rand
    
    # Now create the ranked matrix for running eirene -- it will save us many headaches later.
    edge_list_ranks = denserank([G_i_rand...], rev = true)   # so highest edge weight gets assigned 1
    G_i_rand_ord = reshape(edge_list_ranks,(nNodes,nNodes))
    G_i_rand_ord[diagind(G_i_rand_ord)] .= 0
    weighted_graph_array_iidNoise_ord[:,:,rep] = G_i_rand_ord
    
    
end



    
# Plot an example weighted graph and after we add noise

gr()

test_mat = 2
p0a = heatmap(weighted_graph_array[:,:,test_mat], aspect_ratio = :equal)
p0b = heatmap(weighted_graph_array_iidNoise[:,:,test_mat], aspect_ratio = :equal)
p0c = heatmap(weighted_graph_array_iidNoise_ord[:,:,test_mat], aspect_ratio = :equal)

plot(p0a,p0b,p0c,layout = (1,3))


if save_figures == 1
    savefig("../figures/$(betti_file_name).pdf")
end

if save_data == 1
    save("../processed_data/$(betti_file_name)_graphs.jld",
     "weighted_graph_array", weighted_graph_array,
     "weighted_graph_array_iidNoise", weighted_graph_array_iidNoise,
     "parameters", parameters)
end



Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Thresholding at edge number 846
Threshol

## Running Eirene

In [5]:
# Loop through replicates and run eirene

maxDim = 3
nEdges = binomial(nNodes,2)
barcode_array = Array{Array{Float64}}(undef,nReps,maxDim)
betticurve_array = zeros(nEdges,maxDim,nReps)
beginTime = time()



for rep = 1:nReps
    
    G_i = weighted_graph_array_iidNoise_ord[:,:,rep]
    
    C = Eirene.eirene(G_i,model = "vr", maxdim = maxDim, record = "none")
    println("finished running eirene")
    for k in collect(1:maxDim)
        
        barcode_array[rep,k] = barcode(C,dim=k)
        
    end 
   
    #if rep%10 == 0
        printstyled("\nRun $(rep) finished.\n",color = :cyan)
    #end

    C=0
    endTime = time() - beginTime
    printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

end



endTime = time() - beginTime
printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

if save_data == 1
    save("../processed_data/$(betti_file_name)_eirene_output.jld",
        "barcode_array", barcode_array,
        "betticurve_array", betticurve_array)
end


printstyled("\nDone :)", color = :blue)


finished running eirene

Run 1 finished.
Finished computing persistent homology, elapsed time 10.072552919387817 secondsfinished running eirene

Run 2 finished.
Finished computing persistent homology, elapsed time 10.828411102294922 secondsfinished running eirene

Run 3 finished.
Finished computing persistent homology, elapsed time 11.655718088150024 secondsfinished running eirene

Run 4 finished.
Finished computing persistent homology, elapsed time 11.787775039672852 secondsfinished running eirene

Run 5 finished.
Finished computing persistent homology, elapsed time 12.534999132156372 secondsfinished running eirene

Run 6 finished.
Finished computing persistent homology, elapsed time 13.204632043838501 secondsfinished running eirene

Run 7 finished.
Finished computing persistent homology, elapsed time 13.299566984176636 secondsfinished running eirene

Run 8 finished.
Finished computing persistent homology, elapsed time 14.017811059951782 secondsfinished running eirene

Run 9 finished.

In [10]:


function bettiBarFromBarcode(barcode,nSteps)
    
    bettiBar = 0
    
    for bar in eachrow(barcode)
        
        bar_birth = Int(bar[1])
        bar_death = Int(bar[2])
        
        lifetime = bar_death - bar_birth
        
        bettiBar = bettiBar + lifetime
    end
    
    return bettiBar
end


function muBarFromBarcode(barcode,nSteps)
    
    muBar = 0
    
    for bar in eachrow(barcode)
        
        bar_birth = Int(bar[1])
        bar_death = Int(bar[2])
        
        lifetime_scaled = bar_birth*(bar_death - bar_birth)
        
        muBar = muBar + lifetime_scaled
    end
    
    return muBar
end
    

function nuBarFromBarcode(barcode,nSteps)
    
    # Assuming nSteps is the end of the filtration.
    
    nuBar = 0
    
    for bar in eachrow(barcode)
        
        bar_birth = Int(bar[1])
        bar_death = Int(bar[2])
        
        lifetime_scaled = (nSteps - bar_death)*(bar_death - bar_birth)
        
        nuBar = nuBar + lifetime_scaled
    end
    
    return nuBar
end


nuBarFromBarcode (generic function with 1 method)

In [11]:
# Calculate betti curves from barcodes

beginTime = time()
bettiBar = zeros(maxDim,nReps)
muBar = zeros(maxDim,nReps)
nuBar = zeros(maxDim,nReps)
for rep = 1:nReps
    
    for k in collect(1:maxDim)
        
        betti_curve_i = betticurveFromBarcode(barcode_array[rep,k],nEdges)
        betticurve_array[:,k,rep] = betti_curve_i[1,:]
        
        # Calculate bettiBar, muBar, nuBar
        bettiBar[k, rep] = bettiBarFromBarcode(barcode_array[rep,k],nEdges)
        muBar[k, rep] = muBarFromBarcode(barcode_array[rep,k],nEdges)
        nuBar[k, rep] = nuBarFromBarcode(barcode_array[rep,k],nEdges)
        
        
        
    end 
   
    #if rep%10 == 0
        printstyled("\nRun $(rep) finished.\n",color = :cyan)
    #end


    endTime = time() - beginTime
    printstyled("Finished computing betti curves, elapsed time $(endTime) seconds", color = :cyan)

end


Run 1 finished.
Finished computing betti curves, elapsed time 0.11499714851379395 seconds
Run 2 finished.
Finished computing betti curves, elapsed time 0.11514902114868164 seconds
Run 3 finished.
Finished computing betti curves, elapsed time 0.11525106430053711 seconds
Run 4 finished.
Finished computing betti curves, elapsed time 0.11537599563598633 seconds
Run 5 finished.
Finished computing betti curves, elapsed time 0.11548399925231934 seconds
Run 6 finished.
Finished computing betti curves, elapsed time 0.11559605598449707 seconds
Run 7 finished.
Finished computing betti curves, elapsed time 0.11568999290466309 seconds
Run 8 finished.
Finished computing betti curves, elapsed time 0.11578512191772461 seconds
Run 9 finished.
Finished computing betti curves, elapsed time 0.11591100692749023 seconds
Run 10 finished.
Finished computing betti curves, elapsed time 0.11600804328918457 seconds
Run 11 finished.
Finished computing betti curves, elapsed time 0.11614108085632324 seconds
Run 12 

In [ ]:
## Plot average betti curves
### UPDATE FOR DIM 5

if save_figures == 1
    betticurve_avg = dropdims(mean(betticurve_array,dims=3), dims = 3)
    betticurve_std = dropdims(std(betticurve_array,dims=3), dims = 3)

    x_axis = collect(1:nEdges) ./nEdges
    p1a = plot(x_axis,betticurve_avg[:,1],linewidth = 1.5, ribbon = betticurve_std[:,1], label = "Betti_1")
    for d = 2:maxDim
        plot!(x_axis,betticurve_avg[:,d],linewidth = 1.5, ribbon = betticurve_std[:,d], label = "Betti_$(d)")
    end

    title!("$(betti_file_name)")
    ylabel!("Betti_k")
    xlabel!("Edge Density")

    plot(p1a,layout = (1,1))


    savefig("../figures/$(betti_file_name)_bettis.pdf")
end


In [12]:
bettiBar

3×50 Array{Float64,2}:
 3146.0  1968.0  2263.0   452.0  2214.0  …  1896.0  1852.0  1861.0  2578.0
 2226.0  1832.0  1740.0  2035.0  2611.0     1444.0  1157.0  4113.0  3232.0
 1279.0  3576.0   741.0  2440.0  1969.0      776.0   961.0  4658.0  1516.0

In [16]:
maximum(muBar)

6.694158e6

In [15]:
maximum(nuBar)

5.097115e6